<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению.

### <center> Автор материала: Тышов Никита @nikt

# Keystroke biometrics

## 1. Описание набора данных и признаков

<B> Ценность исследования. </B> 

Очередная задача биометрической идентификации человека. Задача основана на идее о том, что биометрические признаки: отпечаток пальца, лицо, рисунок сетчатки, рисунок радужки и т.д. трудно подделать так как они уникальны и формируются под огромным количеством несвязанных факторов и не поддаются корректировке. Однако, как показало время, технологии не стоят на месте, и параллельно росту качества систем распознавания растет и качество систем взлома. Так биометрические признаки являются устойчивыми довольно длительное время, поэтому их можно попробовать скопировать и подложить, как например с отпечатками пальцев и лицами. Поэтому идут поиски систем на основе поведенческой биометрии. Поведение так же считается уникальным, при этом его довольно трудно скопировать.

Keystroke biometrics рассматривает различные ситуации: верификация человека на основе свободно набираемого текста, или на основе определенно набираемого текста, на основе текста который просит система и т.д. вариаций может быть множество и каждая хороша в своих условиях. Здесь будет рассмотренна распространненая ситуация, когда ваш логин  и пароль был скомпрометирован. То есть, злоумышленник без лишних хлопот может войти в нужный ему аккаунт. Сейчас в основном такая ситуация разрешается при помощи двухфакторной верификации, но это не отменяет попыток найти альтернативы. В данной ситуации двухфакторной верификации нет и необходимо верифицировать пользователя по поведению в наборе пароля.

Также определю два общих для биометрических систем термина: верификация - это когда мы знаем кто стучится, например на основе логина, и пытаемся сказать тот человек или не тот, обычно это задача one-vs-all, то есть бинарная. И идентификация - это когда мы не знаем о человеке ничего кроме его биометрических характеристик и пытаемся найти его в базе или сказать, что такого человека мы вообще не знаем, такая задача почти является классификацией, но классов здесь обычно неограниченное число.

<B>Процесс сбора данных.</B> 
http://www.cs.cmu.edu/~keystroke/DSL-StrongPasswordData.csv

Датасет был собран в <I>Carnegie Mellon University</I> для испытания различных моделей распознавания. Датасет представляет собой тайминг нажатий клавишь на клавиатуре при вводе пароля .tie5Roanl. Пароль был сгенериван случайно и по правилам составляния паролей является сложным так как содержит и знаки препинания и цифры, большие и маленькие буквы. Также пароль имеет приемлемую длину в 10 символов. Основная мотивация при составлении датасета была следующая: предположим что злоумышленнык знает пароль, сможем ли мы идентифицировать его только по стилю набора. 
    
Были вызваны случайные 51 человек, каждый из которых в течении 8 сессий( каждая сессия проходила не чаще чем раз в день) набирал этот пароль 50 раз. Ошибки ввода не принимались, то есть при совершении ошибки необходимо было перенабирать пароль заново. Данные собраны очень грамотно, распределенны во времени и позволяют привыкнуть к паролю, что имитирует ситуацию когда пароль вводится на автомате "мышечной памятью". Следовательно так как пароль один и все испытуемые видели его впервые и набирали в одних и тех же условиях, единственным их отличием должно стать именно поведение.

<B>Целевая переменная.</B> 

Целевой переменной является соответствие личности входящего, личности шаблона или класса с которым его сравнивают.

<B>Описание признаков.</B> 

Как уже было сказано, датасет содержит 51 человека по 8 сессий по 50 наборов, в сумме 20400 записей, и 34 признака.
* subject - id объекта(человека)
* sessionIndex - номер сессии
* rep - номер попытки внутри сессии

Далее идут временный интервалы. Определим:
* key-down : kd - момент нажатия клавиши
* key-up : ku - момент отпускания клавиши
* key - определенная клавиша у нас это . t i e ...

Признаки
* H.key - время удержания key т.е ku.key-kd.key
* DD.key1.key2 - время между kd следующих друг за другом клавишь т.е kd.key2 - kd.key1
* UD.key1.key2 - время между ku первой клавиши и kd следующей т.е kd.key2 - ku.key1



## 2. Первичный анализ данных

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn
import numpy as np

%matplotlib inline

In [ ]:
data = pd.read_csv('../../../DSL-StrongPasswordData.csv')

Посмотрим на данные.

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

В данных нет пропусков. Также можно заметить дополнительные нажатия shift и enter: H.Shift.r, H.Return.

Первые три признака subject, sessionIndex, rep как таковыми признаками не являются, это идентификаторы позволяющие определять принадлежность данных. Далее мы их уберем.

Посмотрим на статистику в данных.

In [ ]:
data.describe()

Для лучшего рассмотрения разобьем описание по типам признаков: отдельно для H, DD и UD

In [ ]:
data[[x for x in data.columns if 'H' in x]].describe()

In [ ]:
data[[x for x in data.columns if 'DD' in x]].describe()

In [ ]:
data[[x for x in data.columns if 'UD' in x]].describe()

Оказывается UD может быть отрицательным, это говорит о том, что человек нажимает следующую клавишу до того как отпустил предыдущую.

Выделим идентификаторы в отдельный датафрейм, чтобы они нам не мешали.

In [ ]:
indx_col = ['subject','sessionIndex','rep']
indx = data[indx_col]
label = [ int(x[1:]) for x in data['subject']]
features = data.drop(columns=indx_col)

## 3. Первичный визуальный анализ данных

Исследуем корреляцию Пирсона числовых признаков.

In [ ]:
a4_dims = (11.7, 8.27)
df = features.corr()
fig, ax = pyplot.subplots(figsize=a4_dims)
seaborn.heatmap(ax=ax, data=df)

По матрице корреляции выявлена сильная зависимость между UD и DD признаками на каждом символе. Посмотрев после этого на данные более пристально, понял, что DD = H + UD. Вспомним, что 
* DD = kd.key2 - kd.key1 
* UD = kd.key2 - ku.key1 
* H = ku.key1 - kd.key1
Удалим DD как избыточный признак, и еще раз посмотрим на матрицу корреляции.

In [ ]:
features_H_UD = features.drop(columns=[x for x in features.columns if 'DD' in x])
df = features_H_UD.corr()
fig, ax = pyplot.subplots(figsize=a4_dims)
seaborn.heatmap(ax=ax, data=df)

Уже лучше. Больше сильных корреляций не наблюдается. Попробуем применить PCA и разложить вектора на плоскости, вдруг все испытуемые хорошо линейно разделимы и лежат обособленно в кластерах.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_H_UD)

plt.figure(figsize=(12,10))
plt.scatter(features_pca[:, 0], features_pca[:, 1], c=label, 
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', 51))
plt.colorbar()

Так просто данные разделить не получилось. Посмотрим какую дисперсию описывают признаки.

In [ ]:
pca = PCA().fit(features_H_UD)
plt.figure(figsize=(10,7))
plt.plot(np.cumsum(pca.explained_variance_ratio_), color='k', lw=2)
plt.xlabel('Number of components')
plt.ylabel('Total explained variance')

Видно, что двух измерений очень мало. С данными необходимо работать как с многомерными векторами.

Визуальный анализ выявил корреляции которые были пропущены при знакомстве с датасетом. Корреляции устранены. Опробован PCA в качестве оценки решения в лоб. Визуально задача в двухмерном пространсве не решается, потому что двух и трех измерений очень мало. 

## 4. Инсайты, найденные зависимости

В целом данные не содержат инстайтов, пробелов и каких либо упущений которые бы сделали данные невалидными. В процессе визуального и первичного анализа выявлены и устранены зависимые переменные и обнаружена не тривиальная оссобенность признаков UD, говорящая о том, что при печати испытуемые могут нажимать следующую клавишу перед тем как отпустили предыдущую.

## 5. Выбор метрики

В качестве основной метрики для оценки качества модели выбран критерий ROC-AUC. Такая метрика хорошо работает с несбалансированными классами. Разбалансированность обычное дело в задачах верификации. В  биометрии ROC кривая является основным показателем качества, так же по нему подбирают пороги при которых насколько возможно минимизируется False positive rate(FPR - ложный пропуск, когда мы пропускаем не того), обычно при уменьшении FAR уменьшается и True positive rate(TPR - пропуск своего, когда мы пропускаем того кого надо) ROC кривая позволяет найти компромисс.

## 6. Выбор модели

Основной задачей является задача верификации, когда нам необходимо определить тот человек или не тот, с учетом того, что мы знаем, кто должен верифицироваться. То есть, это бинарная классификация. Так как у нас 51 человек, получается необходимо будет обучить 51 бинарный классификатор. 

Попробуем решить задачу несколькими методами:
    - метрический, решение принимается на основании расстояния до среднего вектора в классе, будет рассмотрен как линейный классификатор
    - RandomForestClassifier , будет рассмотрен как нелинейный классификатор
    


## 7. Предобработка данных

Данные являются зависимыми во времени, так как пароль вводился в восьми последовательных сеансах. Учтем, что пользователь во время ввода привыкал к паролю и поведение в процессе менялось. Для обучения будем брать первые 3 сеанса верефицируемого пользователя, то есть первые 150 вводов, это не очень правдоподобно(заставить пользователя при регистрации ввести пароль 150 раз не очень гуманно), но и не фантастика. А остальные 250 для теста. В качестве второго класса(атакующих) возьмем по 5 первых последовательных попыток для обучения. А для теста возьмем еще по 5 начиная с 6 с шагом 79, так у нас будет имитация того, что злоумышленник тренируется в набивании пароля.

Так как потребуется обучать много классификаторов, по одному на каждого испытуемого, пересоберем данные в удобный словарь, чтобы было легко их доставать в зависимости от целевого испытуемого. Признаки упакуем в вектор. Так как данные изначально правильно отсортированы это можно сделать последовательно. Дополнительно напишем функцию по разделению данных на классы в зависимости от того какой человек верифицируется.

In [ ]:
from collections import defaultdict

print(len(label), features_H_UD.values.shape)

data_dict = defaultdict(list)

for ind,l in enumerate(label):
    data_dict[l].append(features_H_UD.values[ind])
    
data_len = len(data_dict)
print(data_len)


def prepare_data(data_dict, index):
    train_x = []
    test_x = []
    train_y = []
    test_y = []
    
    for ind, i in enumerate(data_dict):
        if ind==index:
            train_x.extend(data_dict[i][:150])
            train_y.extend([1]*150)
            test_x.extend(data_dict[i][150:])
            test_y.extend([1]*250)
        else:
            train_x.extend(data_dict[i][:5])
            train_y.extend([0]*5)
            test_x.extend(data_dict[i][5::79])
            test_y.extend([0]*5)
    return train_x, train_y, test_x, test_y

trte = prepare_data(data_dict,0)
for i in trte:
    print(np.array(i).shape)

## 8. Кросс-валидация и настройка гиперпараметров модели

Так как настраивать параметры нужно только RandomForest, то на данном этапе работать будем с ним. Параметры будем подбирать на одном испытуемом при помощи кросс-валидации. Переберем большое количество параметров, но параметры сложности и количества деревьев будем искать возле минимальных значений, так как данных очень мало и можно легко переобучиться, возьмем 5 фолдов чтобы сократить время поиска. Не забудем, что метрика ROC-AUC.

In [ ]:
from sklearn.model_selection  import StratifiedKFold
from sklearn.model_selection  import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
RANDOM_STATE = 42
parameters = {'n_estimators':[50,100,200], 'max_features': [4, 7, 10], 
              'min_samples_leaf': [3, 5, 7], 'max_depth': [3,5,10]}

np.random.seed(RANDOM_STATE)
train_x, train_y, test_x, test_y = prepare_data(data_dict,index=np.random.randint(len(data_dict)))


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

rfc = RandomForestClassifier(random_state=RANDOM_STATE,n_jobs=-1)
gcv = GridSearchCV(rfc, parameters, n_jobs=-1, cv=skf, verbose=1,scoring='roc_auc')
gcv.fit(train_x, train_y)

In [ ]:
gcv.best_params_, gcv.best_score_

Кросс валидация выбрала следующие параметры
 - max_depth 10
 - max_features 4
 - min_samples_leaf 3
 - n_estimators 200

значение ROC AUC очень высокое, возможно при тестировании обнаружится переобучение.

In [ ]:
from sklearn.metrics import roc_auc_score
y_pred = gcv.best_estimator_.predict_proba(test_x)
roc_auc_score(test_y,y_pred[:,1])

Тестовая выборка показала небольшое проседание, но не критичное, следовательно найденные параметры можно считать валидными.

## 9. Создание новых признаков и описание этого процесса

Новые признаки создавать фактически не из чего, данные полностью описывают физические процессы. Во втором пункте, при работе с PCA , было замечено, что 10 признаков описывают 99% дисперсии. Попробуем обучить еще один лес на признаках отобранных PCA. 

In [ ]:
rfc_pca = RandomForestClassifier(random_state=RANDOM_STATE,n_jobs=-1)
gcv_pca = GridSearchCV(rfc_pca, parameters, n_jobs=-1, cv=skf, verbose=1,scoring='roc_auc')
train_x_pca = pca.transform(train_x)[:,:10]
gcv_pca.fit(train_x_pca, train_y)

In [ ]:
gcv_pca.best_params_, gcv_pca.best_score_

Параметры оказались теми же, точность упала. Проверим тестовую выборку.

In [ ]:
test_x_pca = pca.transform(test_x)
y_pred_pca = gcv.best_estimator_.predict_proba(test_x_pca)
roc_auc_score(test_y,y_pred_pca[:,1])

На тестовой выборке точность упала очень сильно. Уменьшение количества признаков приводит к уменьшению точности модели.

## 10. Построение кривых валидации и обучения

Построим кривые.

In [ ]:
from sklearn.model_selection import learning_curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    pyplot.figure()
    pyplot.title(title)
    if ylim is not None:
        pyplot.ylim(*ylim)
    pyplot.xlabel("Training examples")
    pyplot.ylabel("Score")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    pyplot.grid()

    pyplot.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    pyplot.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    pyplot.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    pyplot.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    pyplot.legend(loc="best")
    return pyplot



estimator = gcv.best_estimator_
plot_learning_curve(estimator, 'Кривые валидации и обучения', train_x, train_y, ylim=(0.0, 1.01), cv=skf, n_jobs=-1)


pyplot.show()

Модель очень критична к количеству данных. Наблюдается резкий прирост качества при увеличении обучающих примеров.

## 11. Прогноз для тестовой или отложенной выборке

Здесь будем оценивать качество линейного метода и RandomForest на всей выборке. Построим классификатор на основе средних векторов и Евклидова расстояния и рассчитаем его среднюю точность на всей выборке.

In [ ]:
from tqdm import tqdm
all_roc = []
for ind,i in tqdm(enumerate(data_dict)):
    e_train_x, e_train_y, e_test_x, e_test_y = prepare_data(data_dict,ind)
    e_train_x = np.array(e_train_x)  
    e_train_y = np.array(e_train_y) 
    e_test_x = np.array(e_test_x) 
    e_test_y = np.array(e_test_y) 
    target = np.mean(e_train_x[np.where(e_train_y == 1)[0]],axis=0)
    target_norm = target / np.linalg.norm(target)
    e_test_x_normed = e_test_x/ np.linalg.norm(e_test_x,axis=0)[np.newaxis,:]
    predictions = np.sqrt(np.sum((e_test_x_normed - target_norm)**2,axis=1))
    all_roc.append(roc_auc_score(e_test_y,predictions))

np.mean(all_roc)

Линейный подход не сработал результат получается случайный. Пробуем RandomForest c параметрами от кроссвалидации.

In [ ]:
all_roc_forest = []
local_forest = RandomForestClassifier(max_depth=10,max_features=4,min_samples_leaf=3,n_estimators=200,random_state=RANDOM_STATE,n_jobs=-1)

for ind,i in tqdm(enumerate(data_dict)):
    e_train_x, e_train_y, e_test_x, e_test_y = prepare_data(data_dict,ind)
    e_train_x = np.array(e_train_x)  
    e_train_y = np.array(e_train_y) 
    e_test_x = np.array(e_test_x) 
    e_test_y = np.array(e_test_y) 

    
    local_forest.fit(e_train_x,e_train_y)
    predictions = local_forest.predict_proba(e_test_x)[:,1]
    all_roc_forest.append(roc_auc_score(e_test_y,predictions))

np.mean(all_roc_forest)

RandomForest справился намного лучше.Сильного переобучения при подборе параметров не произошло, результаты отличаются не сильно.

## 12. Выводы 

В итоге мы получили хороший результат. Увидели, что линейное сравнение не работает, это говорит о том, что признаки поведения комбинируются сложнее чем рассчитывалось. Хорошо показал себя RandomForest. Был проработан сценарий верификации когда идет сравнение one-vs-all. С помощью случайного леса достигнута средняя ROC-AUC равная 0.9775. Я считаю это хорошим результатом. В данном случае можно сделать вывод о то том, что данная биометрическая верификация работает, и с ней можно работать. Насколько это будет работать в промышленных условиях вопрос открытый, так как там совершенно другие объемы и точность может значительно просесть.

Возможные улучшения:

- увеличение базы для приближения условий к промышленным
- применение нейронных сетей
- применение концепций обучения metric learning

Спасибо за внимание!